In [ ]:
%pip install chess tensorflow keras

In [ ]:
import chess
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import optimizers, losses, callbacks

# Define the evaluation function model
def create_evaluation_model():
    model = keras.Sequential([
        #input layer
        keras.layers.Dense(132, activation='relu', input_shape=[132], kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        
        #hidden layers
        #keras.layers.Dense(4096, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        #keras.layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        #keras.layers.Dropout(0.2),
        #keras.layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        #keras.layers.Dropout(0.2),

        #output layer
        keras.layers.Dense(1, activation='linear')
    ])
    # Set the laerning rate
    learning_rate = 0.01
    
    # Choose the optimizer
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    #optimizer = optimizers.RMSprop(learning_rate=learning_rate)
    #optimizer = optimizers.SGD(learning_rate=learning_rate)
    #optimizer = optimizers.Adagrad(learning_rate=learning_rate)
    
    # Choose the loss function
    #loss=losses.mean_absolute_error
    loss=losses.mean_squared_error
    #loss=losses.binary_crossentropy
    
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [ ]:
# Convert FEN to a feature vector
def fen_to_features(fen):
    board = chess.Board(fen)
    features = np.zeros(132)  # Increase the feature vector size to accommodate additional features
    
    for i in range(64):
        piece = board.piece_at(i)
        if piece is not None:
            piece_type = piece.piece_type
            color = piece.color
            sign = 1 if color == chess.WHITE else -1
            features[i] = sign * piece_type
    
    # Additional features
    features[64] = int(board.turn)  # Current player's turn (0 for black, 1 for white)
    
   # Piece mobility
    for i in range(64):
        piece = board.piece_at(i)
        if piece is not None:
            piece_mobility = len(board.attacks(i))
            features[i + 65] = piece_mobility
    
    # King safety
    white_king_safety = calculate_king_safety(board, chess.WHITE)
    black_king_safety = calculate_king_safety(board, chess.BLACK)
    features[129] = white_king_safety
    features[130] = black_king_safety

    # Forced mate flag
    if board.is_checkmate():
        if board.turn == chess.WHITE and board.result() == "1-0":
            features[131] = 1.0
        elif board.turn == chess.BLACK and board.result() == "0-1":
            features[131] = -1.0
    
    return features
    
def is_forced_mate(eval_score, color):
    if color == chess.WHITE and eval_score >= 4000:
        return True
    elif color == chess.BLACK and eval_score <= -4000:
        return True
    return False    

def calculate_king_safety(board, color):
    king_square = board.king(color)
    king_safety = 0
    
    # Example: Evaluate king safety by counting the number of squares attacked by opponent's pieces
    opponent_color = not color
    for i in range(64):
        if board.is_attacked_by(opponent_color, i):
            king_safety += 1
    
    return king_safety


In [ ]:
def evaluate_position(model, position):
    eval_score = model.predict(np.array([position]))[0][0]
    return eval_score

def is_forced_mate(eval_score, color):
    if color == chess.WHITE and eval_score >= 4000:
        return True
    elif color == chess.BLACK and eval_score <= -4000:
        return True
    return False

# Read training data from CSV
training_data = pd.read_csv('/kaggle/input/chesstraindata/chessData-10L.csv')

# Convert training data to feature vectors
training_positions = []
training_scores = []

for i in range(len(training_data['FEN'])):
    fen = training_data['FEN'][i]
    position = fen_to_features(fen)
    score = float(training_data['eval'][i])
    
    if not is_forced_mate(score, chess.WHITE) and not is_forced_mate(score, chess.BLACK):
        training_positions.append(position)
        training_scores.append(score)

In [ ]:
# Create and train the evaluation function model
model = create_evaluation_model()

# Define the ProgbarLogger callback
#progbar_logger = keras.callbacks.ProgbarLogger()

# Fit the model with the ProgbarLogger callback
#model.fit(np.array(training_positions), np.array(training_scores), epochs=500, batch_size=1048575, callbacks=[progbar_logger])
model.fit(np.array(training_positions), np.array(training_scores), epochs=35, batch_size=20972)

In [ ]:
# save and load the model
model.save('DeepCarlsen.h5')
#model = keras.models.load_model('my_model.h5')
model = keras.models.load_model('DeepCarlsen.h5')

In [ ]:
# Test example
test_example = 'r2q1rk1/p1p1bppp/2p2n2/3p2B1/4P1b1/2NB1Q2/PPP2PPP/2KR3R w - - 8 11'
# A position from a game played by me as black with eval = -1.17, although both side have equal values of pieces

# Convert test example to feature vector
test_position = fen_to_features(test_example)

# Predict the evaluation score for the test example
evaluation_score = evaluate_position(model, test_position)

print('Test Example:', test_example)
print('Evaluation Score:', evaluation_score)

In [ ]:
def minimax(position, depth, maximizing_player, evaluate_position):
    if depth == 0 or position.is_game_over():
        return evaluate_position(model, position)
    
    if maximizing_player:
        max_eval = float('-inf')
        for move in position.legal_moves:
            position.push(move)
            eval_score = minimax(position, depth - 1, False, evaluate_position)
            position.pop()
            max_eval = max(max_eval, eval_score)
        return max_eval
    else:
        min_eval = float('inf')
        for move in position.legal_moves:
            position.push(move)
            eval_score = minimax(position, depth - 1, True, evaluate_position)
            position.pop()
            min_eval = min(min_eval, eval_score)
        return min_eval


def choose_best_move(position, depth):
    best_move = None
    best_eval = float('-inf')
    for move in position.legal_moves:
        position.push(move)
        eval_score = minimax(position, depth - 1, False, evaluate_position)
        position.pop()
        if eval_score > best_eval:
            best_eval = eval_score
            best_move = move
    return best_move

# Example usage
best_move = choose_best_move(test_position, depth=3)
print("Best Move :", best_move)
